## Setup

In [ ]:
from functools import reduce

import mcl
mcl.mcl_init(mcl.CurveType.MCL_BLS12_381)

Q = mcl.G1.BLS12_381_G1_generator()

## User

In [ ]:
password = 'cl0udc0mp'
fid = 'my_file.txt'

In [ ]:
# Create secret key based on password
sk = mcl.Fr()
sk = sk.set_hash_of(password)
print('Sk:', sk)

In [ ]:
# Encode file into chunks

m_a_user = []
with open(fid, "rb") as file:
    while True:
        data = file.read(31)
        print(data)
        if not data:
            break
        f = mcl.Fr()
        f.setStr(str(int(data.hex(), base=16)))
        m_a_user.append(f)

In [ ]:
print('m_i encoded chunks:', *m_a_user, sep='\n')

In [ ]:
# Create polynomial L
L_coef = [mcl.Fr().set_hash_of(f'{sk}{fid}{i}') for i in range(len(m_a_user))]
polynomial_core = lambda c_a: lambda x: reduce(lambda acc, val: acc * x + val, c_a)
assert(polynomial_core([1, 2, 3])(5) == 1 * 5 * 5 + 2 * 5 + 3)
L = polynomial_core(L_coef)

# print('L coefficients:', *L_coef, sep='\n')

In [ ]:
# Create tags for chunks
t_a_user = [L(m) for m in m_a_user]
print('Pairs (m_i, t_i):', *list(zip(m_a_user, t_a_user)), sep='\n')

## Cloud

In [ ]:
m_a_cloud = list(m_a_user)
t_a_cloud = list(t_a_user)

## Challenge

In [ ]:
# Generate random generator
r = mcl.Fr()
r.set_by_CSPRNG()
Qr = mcl.G1()
Qr = Q * r
print('Qr:', Qr)

In [ ]:
# Generate challenge
x_c = mcl.Fr()
while True:
    x_c.set_by_CSPRNG()
    if x_c not in m_a_user:
        break
print('x_c:', x_c)

In [ ]:
# Calculate Qr ^ L(0)
QrL0 = mcl.G1()
QrL0 = Qr * L(mcl.Fr())
print('QrL0:', QrL0)

In [ ]:
# Calculate expected response R_u = Qr ^ L(x_c)
R_u = Qr * L(x_c)
print('R_u:', R_u)

## Response

In [ ]:
Qr_cloud = Qr
x_c_cloud = x_c
QrL0_cloud = QrL0

In [ ]:
# Prepare interpolation set
m_a_cloud_tmp = list(m_a_cloud)
t_a_cloud_tmp = [Qr_cloud * t for t in t_a_cloud]
m_a_cloud_tmp.append(mcl.Fr())
t_a_cloud_tmp.append(QrL0)
print('Pairs (m_i, t*_i):', *list(zip(m_a_cloud_tmp, t_a_cloud_tmp)), sep='\n')

In [ ]:
# Interpolate L with use of LI_EXP, calculate given response R_c = LI_EXP(x_c)

# Get m_a_cloud_tmp array without an element on given index i
filter_i = lambda a: lambda i: [val for ind, val in enumerate(a) if ind != i]
assert(filter_i([1, 2, 3, 4])(2) == [1, 2, 4])

LI_core = lambda x_a, i, s_0: lambda x: reduce(lambda acc, val: ((x - val) / (x_a[i] - val)) * acc, filter_i(x_a)(i), s_0)
assert(LI_core([1, 2, 3], 1, 1)(5) == ((5 - 1) / (2 - 1)) * (5 - 3) / (2 - 3))

LI_EXP = lambda x_a, y_a, s_0, s_1: lambda x: reduce(lambda acc, val: (val[1] * LI_core(x_a, val[0], s_0)(x)) + acc, enumerate(y_a), s_1)
FR_ONE = mcl.Fr()
FR_ONE.setInt(1)
R_c = LI_EXP(m_a_cloud_tmp, t_a_cloud_tmp, FR_ONE, mcl.G1())(x_c)

print('R_c:', R_c)

## Verify

In [ ]:
if R_u == R_c:
    print('Verified')
else:
    print('Rejected')

## Download

In [ ]:
m_a_download = list(m_a_cloud)

In [ ]:
# Decode downloaded blocks
with open("dec_" + fid, "wb") as file:
    for m in m_a_download:
        byte_data = int(m.getStr()).to_bytes(31, byteorder='big')
        print(byte_data.replace(b'\x00', b''))